In [ ]:
#loss = bias + variance + noise
#use for regression perfomance measure

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from cuml.manifold import TSNE

In [ ]:
trainDF = pd.read_csv('../app/data/02_train.csv', sep=';')
testDF = pd.read_csv('../app/data/02_test.csv', sep=';')

In [ ]:
X_train = trainDF.drop(['totalPrice'], axis=1)
y_train = trainDF['totalPrice']
X_test = testDF.drop(['totalPrice'], axis=1)
y_test = testDF['totalPrice']

In [ ]:
X_train

In [ ]:
embedded = TSNE(n_components = 2).fit_transform(X_train[['Lng', 'Lat']])

In [ ]:
fig, ax = plt.subplots(figsize=(18, 18))
ax.scatter(embedded[:,0], embedded[:,1], s=0.1, c=embedded[:,0], cmap=plt.cm.plasma)
ax.set_title('t-SNE plot for all 130 features', fontsize=18)
plt.xlim([-500, 500])
plt.ylim([-500, 500])
plt.show();